In [1]:
import os
import re
from Tools_r2 import calculate, currency_converter, get_news, ddg_search
from jinja2 import Template

from IPython.display import Image, display

### We will breakdown the prompt into 3 parts and then join for easy understanding.
1. System prompt (main prompt, mentioning the Agents role and teach ReAct methodology)
2. Actions prompt (Tools description, and parameters they take)
3. Chain of Thought Example

#### System prompt

In [2]:
role = 'You are an intelligent agent capable of answering questions and performing tasks using available tools:'
name = "Assistant"
# Defining the Jinja2 template for the system prompt
system_prompt_template = """
Your name is: {{name}}
Your role is: {{role}}.

{% if tools %}
Your available tools are:
{% for tool_name, tool_func in tools.items() %}
{{ tool_name }}:
Usage: {{ tool_func.__doc__.strip() }}
{% endfor %}
{% endif %}

Always Respond in this format:
Thought: Your reasoning about the task
Action: tool_name: arguments
PAUSE

After receiving an observation, continue with:
Thought: Your interpretation of the observation
Action: Next action or "Answer" if you have the final response

Answer: Your final response to the user's query
{% if cot_example %}
'''
Chain of Thought Example(s): 
    {{ cot_example }}'''
{% endif %}

Begin!
"""

#### Chain of Thought Example

In [3]:
cot_example = """
Question: What is USD 1 equivalent in GBP?
Thought: I need to find the currency conversion rate for 1 USD to GBP. I need to call the currency_converter
Action: currency_converter: 1.0, "USD", "GBP"
PAUSE
Observation: 1 USD is equal to 0.8 GBP"
"""

### Testing system prompt

In [4]:
tools = {"calculate": calculate, "currency_converter": currency_converter, 'ddg_search': ddg_search, 'get_news': get_news}
system_prompt = Template(system_prompt_template).render(role=role, tools=tools, cot_example=cot_example)

print(system_prompt)


Your name is: 
Your role is: You are an intelligent agent capable of answering questions and performing tasks using available tools:.


Your available tools are:

calculate:
Usage: Evaluate a mathematical expression and return the result as a float.    
    :param expression: A string representing the mathematical expression to evaluate.
    :return: The result of the evaluation as a float. If an error occurs during evaluation, returns NaN.

currency_converter:
Usage: Converts an amount from a source currency to a target currency.
    :param amount: The amount in the source currency.
    :param source_curr: The source currency code (e.g., 'USD').
    :param target_curr: The target currency code (e.g., 'EUR').
    :return: A formatted string with the converted amount.
    '''Example: currency_converter(100, 'USD', 'EUR')
    '100.00 USD is equivalent to: 80.00 EUR' '''

ddg_search:
Usage: Search DuckDuckGo for a query and return the results.

    :param query: The query to search for.


#### For Agents we need to maintain a state of messages from begining till last until answer is evaluated.

The Chat Models has a dictionary style prompt, have 'role' keys and 'content' keys.

We need to maintain state between Agent cycles of Thought, Action, Pause, Observation 

### Agent Class

In [5]:
class Agent:
    def __init__(self, llm: any, name: str=name, role:str=role, system_prompt: str = system_prompt_template,
                tools: dict = {}, cot_example: str = None, max_iterations=20) -> None:
        """
        Initializes the Agent with an LLM instance and an optional system message.
        :param llm: An instance of the LLM class used to generate responses.
        :param system_prompt: An optional system message to initialize the conversation context.
        """
        self.llm = llm
        self.tools = tools or {}
        self.system = Template(system_prompt).render(name=name, role=role, cot_example=cot_example, tools=self.tools)
        self.messages_state = [{"role": "system", "content": self.system}]
        self.max_iterations = max_iterations
        
    def __call__(self, message=""):
        """
        Handle incoming user messages and generate a response using the LLM.
        
        :param message: The user's message to be added to the conversation.
        :return: The assistant's response to the user.
        """
        if message:
            self.messages_state.append({"role": "user", "content": message})
        result = self.llm.generate_response(self.messages_state)
        self.messages_state.append({"role": "assistant", "content": result})
        return result

    def run(self, query, max_iterations=10):
        print(f"User: {query}")
        next_prompt = query

        for i in range(self.max_iterations):
            #call the agent and update states with user query and agent response
            result = self(next_prompt) # self.__call__(next_prompt)
            print(f'Iter:{i}.\nAgent: {result}')
            # check of previous system response contains words 'PAUSE' and 'Action'
            if "PAUSE" in result and "Action" in result:
                # get word 'Action' with succeeding text (contains tool name and arguments)
                action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
                print('Action:', action)
                
                if action:
                    chosen_tool = action[0][0]  # get tool name
                    arg_str = action[0][1]      # get tool arguments
                    #print('Chosen Tool:', chosen_tool, 'with arguments', arg_str)
                    
                    if chosen_tool in self.tools: # checks if tool exists
                        # Split arguments and strip any unnecessary spaces or quotes
                        arg_list = [arg.strip().strip("'\"") for arg in arg_str.split(",")]
                        print(arg_list)
                        
                        try: # try to execute tool
                            print(f'Calling: {chosen_tool}({arg_str})', end='')
                            result_tool = tools[chosen_tool](*arg_list)
                            observation = f"\nObservation: {result_tool}"
                            
                        except Exception as e:
                            observation = f"\nObservation: Failed to execute tool '{chosen_tool}'. Error: {str(e)}"
                    else:
                        observation = f"\nObservation: Tool '{chosen_tool}' not found"

                    next_prompt = observation # observation becomes next prompt
                    print(next_prompt, '\n')
                    continue

            # check after each iteration if previous system response contains word 'Answer'
            if "Answer" in result: 
                break # break the loop and move to next cell

### Instantiating and Querying the Agent

In [6]:
from Models_r1 import GroqLLM
import os
os.system('clear')

llm = GroqLLM(api_key=os.environ.get('GROQ_API_KEY'))
tools = {
    "ddg_search": ddg_search,
    "currency_converter": currency_converter,
    "get_news": get_news
}

agent = Agent(llm, tools=tools, cot_example=cot_example, max_iterations=20)
query = """"What's are the top 10 latest news about AI (list in bullets)
and how much is 100 USD in EUR? """

agent.run(query)

User: "What's are the top 10 latest news about AI (list in bullets)
and how much is 100 USD in EUR? 
Iter:0.
Agent: Thought: To fulfill this query, I need to perform two separate tasks: first, I'll search for the latest news on AI using the get_news tool, and then I'll convert 100 USD to EUR using the currency_converter tool.

Action: get_news: "AI", 10
Action: currency_converter: 100.0, "USD", "EUR"
PAUSE
Action: [('get_news', '"AI", 10'), ('currency_converter', '100.0, "USD", "EUR"')]
['AI', '10']
Calling: get_news("AI", 10)
Observation: [
  {
    "date": "2024-10-19T03:27:00+00:00",
    "title": "The Age of AI Child Abuse Is Here",
    "body": "For maybe the first time, the scale of the problem is coming into view.",
    "url": "https://www.theatlantic.com/technology/archive/2024/10/muah-ai-hack-child-abuse/680300/",
    "image": "https://cdn.theatlantic.com/thumbor/u-cTwQ107KG3KExlWGfQKbr5sX0=/0x43:2000x1085/1200x625/media/img/mt/2024/10/muah_ai/original.jpg",
    "source": "The At